# Data Preparation

In [50]:
# from google.colab import drive
# drive.mount('/content/drive')

# Evaluation

In [51]:
# import package for displaying NER results
from spacy import load, displacy
import pandas as pd
import joblib
import re
import time
import datetime
from datetime import datetime as dt

In [52]:
kodePos= pd.read_csv('dataset/lokasi2.csv')
kodePos.head()

,provinsi,kabupaten,kecamatan
0,bali,badung,kuta selatan
1,bali,badung,kuta
2,bali,badung,kuta utara
3,bali,badung,petang
4,bali,badung,abiansemal


In [53]:
listLokasi=[]
for x in range(0,len(kodePos)):
    arr=[kodePos.iloc[x,0],kodePos.iloc[x,1],kodePos.iloc[x,2]]
    listLokasi.append(arr)

In [54]:
import joblib

joblib.dump(listLokasi,"listProvKabKec.pkl")

['listProvKabKec.pkl']

In [55]:
def compareToDatasetLocation(provinsi,kabupaten,kecamatan):
      trueLocation= checkLocation(provinsi,kabupaten,kecamatan)
      print(trueLocation)
      if(len(trueLocation)==3):
            for i in range(0,len(kodePos)):
                  if(kodePos.iloc[i,0]==trueLocation[2] and kodePos.iloc[i,1]==trueLocation[1] and kodePos.iloc[i,2]==trueLocation[0]):
                        return trueLocation
                  
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation

      elif(len(trueLocation)==2):
            for i in range(0,len(kodePos)):
                  if(kodePos.iloc[i,0]==trueLocation[1] and kodePos.iloc[i,1]==trueLocation[0]):
                        trueLocation.insert(0,'-')
                        return trueLocation
                  elif(kodePos.iloc[i,1]==trueLocation[1] and kodePos.iloc[i,2]==trueLocation[0]):
                        trueLocation.insert(2,'-')
                        return trueLocation
                  elif(kodePos.iloc[i,0]==trueLocation[1] and kodePos.iloc[i,2]==trueLocation[0]):
                        trueLocation.insert(1,'-')
                        return trueLocation
      
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation

      elif(len(trueLocation)==1):
            for i in range(0,len(kodePos)):
                  if(kodePos.iloc[i,0]==trueLocation[0] ):
                        trueLocation.insert(0,'-')
                        trueLocation.insert(1,'-')
                        return trueLocation
                  elif(kodePos.iloc[i,1]==trueLocation[0] ):
                        trueLocation.insert(0,'-')
                        trueLocation.insert(2,'-')
                        return trueLocation
                  elif(kodePos.iloc[i,2]==trueLocation[0] ):
                        trueLocation.insert(1,'-')
                        trueLocation.insert(2,'-')
                        return trueLocation
      
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation
            
      else:
            trueLocation.insert(0,'-')
            trueLocation.insert(1,'-')
            trueLocation.insert(2,'-')
            return trueLocation


In [56]:
def checkLocation(provinsi,kabupaten,kecamatan):
    hasil=[]
    if(len(provinsi)>0 and len(kabupaten)>0 and len(kecamatan)>0):
        provinsi_top=provinsi[0]
        kabupaten_top=kabupaten[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<kabupaten_top[1]<provinsi_top[1]):
            if(kabupaten_top[1]-kecamatan_top[2]<=60 and provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            elif(kabupaten_top[1]-kecamatan_top[2]<=60 and provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
            elif(kabupaten_top[1]-kecamatan_top[2]>60 and provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append((kecamatan_top[0]))
        elif((kecamatan_top[1]<kabupaten_top[1])and(kabupaten_top[1]>provinsi_top[1])and(kecamatan_top[1]<provinsi_top[1])):
            if(provinsi_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(provinsi_top[0])
            elif(provinsi_top[1]-kecamatan_top[2]>60):
                hasil.append(kecamatan_top[0])
        elif((kecamatan_top[1]>kabupaten_top[1])and(kabupaten_top[1]<provinsi_top[1])and(kecamatan_top[1]>provinsi_top[1])):
            if(provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            elif(provinsi_top[1]-kabupaten_top[2]>60):
                hasil.append((kabupaten_top[0]))
        elif((kecamatan_top[1]>kabupaten_top[1])and(kabupaten_top[1]<provinsi_top[1])and(kecamatan_top[1]<provinsi_top[1])):
            hasil.append(kabupaten_top[0])
        elif((kecamatan_top[1]<kabupaten_top[1])and(kabupaten_top[1]>provinsi_top[1])and(kecamatan_top[1]>provinsi_top[1])):
            hasil.append(provinsi_top[0])
        elif(kecamatan_top[1]>kabupaten_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)>0 and len(kabupaten)>0 and len(kecamatan)==0):
        provinsi_top=provinsi[0]
        kabupaten_top=kabupaten[0]
        if(kabupaten_top[1]<provinsi_top[1]):
            if(provinsi_top[1]-kabupaten_top[2]<=60):
                hasil.append(kabupaten_top[0])
                hasil.append(provinsi_top[0])
            else:
                hasil.append(kabupaten_top[0])
        elif(kabupaten_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)>0 and len(kabupaten)==0 and len(kecamatan)>0):
        provinsi_top=provinsi[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<provinsi_top[1]):
            if(provinsi_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(provinsi_top[0])
            else:
                hasil.append(kecamatan_top[0])
        elif(kecamatan_top[1]>provinsi_top[1]):
            hasil.append(provinsi_top[0])
    elif(len(provinsi)==0 and len(kabupaten)>0 and len(kecamatan)>0):
        kabupaten_top=kabupaten[0]
        kecamatan_top=kecamatan[0]
        if(kecamatan_top[1]<kabupaten_top[1]):
            if(kabupaten_top[1]-kecamatan_top[2]<=60):
                hasil.append(kecamatan_top[0])
                hasil.append(kabupaten_top[0])
            else:
                hasil.append(kecamatan_top[0])
        elif(kecamatan_top[1]>kabupaten_top[1]):
            hasil.append(kabupaten_top[0])
    elif(len(provinsi)>0 and len(kabupaten)==0 and len(kecamatan)==0):
        hasil.append(provinsi[0][0])
    elif(len(provinsi)==0 and len(kabupaten)>0 and len(kecamatan)==0):
        hasil.append(kabupaten[0][0])
    elif(len(provinsi)==0 and len(kabupaten)==0 and len(kecamatan)>0):
        hasil.append(kecamatan[0][0])
    elif(len(provinsi)==0 and len(kabupaten)==0 and len(kecamatan)==0):
        return hasil
    
    return hasil



In [57]:
# import model
link_to_model = "D:/HERRI/Tugas Akhir/pra/geographical_news_scrapper/NER/NER_Kec_Ben_Krim_Kese_Eko/output/model-last"
loaded_model = load(link_to_model)

In [58]:
test_text = """beliau pulang dinas aceh melewati jalan lokasi penembakan kabid humas bandung polda maluku kombes Muhammad Roem ohoirat roem bharaka tewas ditembak jalan mengaku mengkonfirmasi bharaka ditembak berkendara jalan ditembak situlah versi situ berboncengan versi roem roem ditangani pom ad penjelasan terkait kronologi penembakan memeriksa pom ad diberitakan menembak bharaka pratu ri menembak rekannya anggota tni prada prada dilaporkan kritis meninggal kondisi kritis rumah sakit di aceh barat meninggal anggota brimob kapendam xvi pattimura kolonel arh adi prayogo choirul fajar wartawan rabu 163 penembakan anggota tni polri rabu 163 pratu ri menjalani pemeriksaan intensif polisi militer motif penembakan pratu ri di jakarta barat diduga mengalami depresi bandar pasir mandoge indikasinya kondisi depresi akut pemeriksaan psikologis barat kejiwaan didalami diperiksa"""

In [59]:
doc = loaded_model(test_text)
displacy.render(doc, style="ent", jupyter=True)

In [60]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

aceh 20 24 LOKASI
maluku 84 90 LOKASI
adi prayogo 551 562 ORANG
jakarta barat 717 730 LOKASI


In [61]:
df_test = pd.read_csv('D:/HERRI/Tugas Akhir/pra/geographical_news_scrapper/NER/NER_Kec_Ben_Krim_Kese_Eko/dataset/df_test_lower_sof.csv')
df_test.head()

,title,date,description,source
0,"Istri Idap Diabetes, Warga Magelang Bersyukur ...","Senin, 29 Nov 2021 15:18 WIB",\- warga kecamatan tempuran magelang sekaligus...,www.detik.com
1,"Pengendara Mobil Kejang-kejang di Tol Sunter, ...","Jumat, 26 Jun 2020 05:43 WIB",\- seorang pria pengendara mobil ditemukan kej...,www.detik.com
2,Ma'ruf Ungkap Pentingnya Sertifikat Pranikah: ...,"Jumat, 22 Nov 2019 17:38 WIB",\- wakil presiden ma'ruf amin menegaskan komit...,www.detik.com
3,Ma'ruf Kumpulkan Para Menteri Bahas Strategi P...,"Selasa, 11 Feb 2020 09:52 WIB",\- wakil presiden ma'ruf amin memimpin rapat t...,www.detik.com
4,Wali Kota Mojokerto Punya Cara Unik Jaga Keseh...,"Senin, 18 Apr 2022 16:23 WIB",\- wali kota mojokerto ika puspitasari punya c...,www.detik.com


In [62]:
lokasi_csv = 'D:/HERRI/Tugas Akhir/pra/geographical_news_scrapper/NER/NER_Kec_Ben_Krim_Kese_Eko/dataset/location.csv'
df_lokasi = pd.read_csv(lokasi_csv)
df_lokasi.head()

,nama,lokasi
0,bali,provinsi
1,bangka belitung,provinsi
2,banten,provinsi
3,bengkulu,provinsi
4,di yogyakarta,provinsi


In [63]:
list_provinsi=joblib.load('dataset/modelLokasi/provinsi.pkl')
list_kabupaten=joblib.load('dataset/modelLokasi/kabupaten.pkl')
list_kecamatan=joblib.load('dataset/modelLokasi/kecamatan.pkl')


In [64]:
def getNewsTime(dateTime: str):
    month_to_number = {
      'jan': '1',
      'feb': '2',
      'mar': '3',
      'apr': '4',
      'mei': '5',
      'jun': '6',
      'jul': '7',
      'agu': '8',
      'sep': '9',
      'okt': '10',
      'nov': '11',
      'des': '12',
    }

    splitted_timestamp = dateTime.split()
    pre_formatted_time = splitted_timestamp[1] + '/' + month_to_number[splitted_timestamp[2].lower()] + '/' + splitted_timestamp[3]
    
    return pre_formatted_time

In [72]:
def compareDateArray(sortByLength,newsDate):
    print("newsDate : ", newsDate)
    print("sortByLength : ", sortByLength)
    year = newsDate.split('/')[2]
    timestampList = []
    
    def convertToTimestapms(date):
        return int(time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y").timetuple()))

    for i, tempTime in enumerate(sortByLength):
        # Find the content inside parentheses
        print("tempTime : ", tempTime)
        if(tempTime[-1:]!=')'):
            tempTime=tempTime+')'
            
        currTime = re.search(r'\((.*?)\)', tempTime)
        
        if (currTime != None):
            currTime = currTime.group(1)
            currTime = str(currTime)
            currTime=currTime.replace("\\", "")
        
            
        print("currTime : ", currTime,type(currTime))
        
        if (tempTime.lower() == 'kemarin' or ('kemarin' in tempTime.lower())):
            timestampList.append(convertToTimestapms(newsDate) - (24*60*60))
        
        if (currTime == None or currTime==''):
            
            return newsDate
        else:
            # Find the year
            print("currTime2 : ", currTime)
            splitted = currTime.split('/')
            print("splitted : ", splitted)

            formattedDate = None
            if (len(splitted) == 3):
                if(len(splitted[2])==2):
                    splitted[2]=year
                    currTime="/".join(splitted)
                year = splitted[2]
                timestampList.append(convertToTimestapms(currTime))
            elif (len(splitted) == 2):
                formattedDate = (splitted[0] + '/' + splitted[1] + '/' + year)
                timestampList.append(convertToTimestapms(formattedDate))
            
    print(timestampList)
    return dt.fromtimestamp(sorted(timestampList)[0]).strftime('%d/%m/%Y')

In [66]:
def toOriginal(teks):
    hasil=''
    if(teks=='sulsel'):
        hasil='sulawesi selatan'
    elif(teks=='sultra'):
        hasil='sulawesi tenggara'
    elif(teks=='sulut'):
        hasil='sulawesi utara'
    elif(teks=='sulteng'):
        hasil='sulawesi tengah'
    elif(teks=='sulbar'):
        hasil='sulawesi barat'
    elif(teks=='kaltim'):
        hasil='kalimantan timur'
    elif(teks=='kaltara'):
        hasil='kalimantan utara'
    elif(teks=='kalteng'):
        hasil='kalimantan tengah'
    elif(teks=='kalsel'):
        hasil='kalimantan selatan'
    elif(teks=='kalbar'):
        hasil='kalimantan barat'
    elif(teks=='jatim'):
        hasil='jawa timur'
    elif(teks=='jateng'):
        hasil='jawa tengah'
    elif(teks=='jabar'):
        hasil='jawa barat'
    elif(teks=='diy'):
        hasil='di yogyakarta'
    elif(teks=='jakbar'):
        hasil='jakarta barat'
    elif(teks=='jaktim'):
        hasil='jakarta timur'
    elif(teks=='jaksel'):
        hasil='jakarta selatan'
    elif(teks=='jakpus'):
        hasil='jakarta pusat'
    elif(teks=='jakut'):
        hasil='jakarta utara'
    elif(teks=='nad'):
        hasil='nanggroe aceh darussalam'
    elif(teks=='sumsel'):
        hasil='sumatera selatan'
    elif(teks=='sumut'):
        hasil='sumatera utara'
    elif(teks=='sumbar'):
        hasil='sumatera barat'
    else:
        hasil=teks
    return hasil

In [73]:
document_results = []
for i in range(0, df_test.shape[0]):
# for i in range(0, 1):
      print(i)
      news = df_test.iloc[i, -2]
      doc = loaded_model(news)

      # newsDate = getNewsTime(df_test.iloc[21,1])
      newsDate = getNewsTime(df_test.iloc[i,1])

      provinsi = []
      kabupaten = []
      kecamatan = []
      tanggal = []
      orang = []
      visited = []

      tanggal_asli=''

      for ent in doc.ents:
            if(ent.label_== 'LOKASI'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        teks = toOriginal(teks)
                        if ((teks in list_provinsi) and (teks not in visited)):
                              provinsi.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kabupaten) and (teks not in visited)):
                              kabupaten.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kecamatan) and (teks not in visited)):
                              kecamatan.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                  else:
                        teks = toOriginal(ent.text)
                        if ((teks in list_provinsi) and (teks not in visited)):
                              provinsi.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kabupaten) and (teks not in visited)):
                              kabupaten.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
                        elif ((teks in list_kecamatan) and (teks not in visited)):
                              kecamatan.append([teks,ent.start_char,ent.end_char])
                              visited.append(teks)
            elif(ent.label_== 'TANGGAL'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              tanggal.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              tanggal.append(ent.text)
                              visited.append(ent.text)
            elif(ent.label_== 'ORANG'):
                  if(ent.text[-1:]=='.'):
                        teks = ent.text[:-1]
                        if (teks not in visited):
                              orang.append(teks)
                              visited.append(teks)
                  else:
                        if (ent.text not in visited):
                              orang.append(ent.text)
                              visited.append(ent.text)

      visited.clear()

      sortedTimes = sorted(tanggal, key=len, reverse=True)
      # if array is empty return news date
      if(len(sortedTimes) == 0):
            tanggal_asli=newsDate
            # print(newsDate)
      # if array not empty and not contain kemarin
      elif (len(sortedTimes) >= 1):
            tanggal_asli=compareDateArray(sortedTimes,newsDate)
            print(tanggal_asli)
            # print(compareDateArray(sortedTimes,newsDate))

      lokasi2 = compareToDatasetLocation(provinsi,kabupaten,kecamatan)

      
      if(len(orang)==0):
            trueTopik=0
      else:
            trueTopik=1
      if(tanggal_asli is not None):
            trueTanggal=1
      else:
            trueTanggal=0
      if(lokasi2[2]=='-' and lokasi2[1]=='-' and lokasi2[0]=='-'):
            trueLokasi=0
      else:
            trueLokasi=1

      document_results.append([
      orang,
      trueTopik,
      tanggal_asli,
      trueTanggal,
      df_test.iloc[i,1],
      lokasi2[2],
      lokasi2[1],
      lokasi2[0],
      trueLokasi,
      '',
      '',
      '',
      news,
      ])

writer = pd.DataFrame(document_results, columns=['Who','True Who','When','True When','date','Provinsi','Kabupaten','Kecamatan','True Where', 'True Positif', 'True Negative', 'Relevansi Berita', 'Skimming News'])
writer.to_excel('D:/HERRI/Tugas Akhir/pra/geographical_news_scrapper/NER/NER_Kec_Ben_Krim_Kese_Eko/result/result_sof2.xlsx', index=False)

0
newsDate :  29/11/2021
sortByLength :  ['senin (29/11/2021)']
tempTime :  senin (29/11/2021)
currTime :  29/11/2021 <class 'str'>
currTime2 :  29/11/2021
splitted :  ['29', '11', '2021']
[1638118800]
29/11/2021
['tempuran']
1
newsDate :  26/6/2020
sortByLength :  ['jumat (26/6/2020)']
tempTime :  jumat (26/6/2020)
currTime :  26/6/2020 <class 'str'>
currTime2 :  26/6/2020
splitted :  ['26', '6', '2020']
[1593104400]
26/06/2020
['jakarta utara']
2
newsDate :  22/11/2019
sortByLength :  ['jumat (22/11/2019)']
tempTime :  jumat (22/11/2019)
currTime :  22/11/2019 <class 'str'>
currTime2 :  22/11/2019
splitted :  ['22', '11', '2019']
[1574355600]
22/11/2019
['cirebon', 'jawa barat']
3
newsDate :  11/2/2020
sortByLength :  ['selasa (11/2/2020)']
tempTime :  selasa (11/2/2020)
currTime :  11/2/2020 <class 'str'>
currTime2 :  11/2/2020
splitted :  ['11', '2', '2020']
[1581354000]
11/02/2020
['jakarta pusat']
4
newsDate :  18/4/2022
sortByLength :  ['senin (18/4/2022)']
tempTime :  senin (18

In [ ]:
# PERSON:      People, including fictional.
# NORP:        Nationalities or religious or political groups.
# FAC:         Buildings, airports, highways, bridges, etc.
# ORG:         Companies, agencies, institutions, etc.
# GPE:         Countries, cities, states.
# LOC:         Non-GPE locations, mountain ranges, bodies of water.
# PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
# EVENT:       Named hurricanes, battles, wars, sports events, etc.
# WORK_OF_ART: Titles of books, songs, etc.
# LAW:         Named documents made into laws.
# LANGUAGE:    Any named language.
# DATE:        Absolute or relative dates or periods.
# TIME:        Times smaller than a day.
# PERCENT:     Percentage, including ”%“.
# MONEY:       Monetary values, including unit.
# QUANTITY:    Measurements, as of weight or distance.
# ORDINAL:     “first”, “second”, etc.
# CARDINAL:    Numerals that do not fall under another type.